In [69]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [70]:
# столбцы, удаляемые за ненадобностью в обучении
drop_cols = []

# столбцы категорий
cat_cols = [
    "Размер компании.Наименование",
    "ОКВЭД2.Наименование", "Город фактический",
    "Город юридический", "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание",
    "Карточка клиента (внешний источник).Индекс финансового риска Описание",
    "Госконтракты.Контракт", "Госконтракты.Тип контракта"
]

# столбцы числовых значений
num_cols = [
    "Размер уставного капитала объявленный",
    "ОКВЭД2.Код", "Численность персонала по данным ФНС.Количество",
    "Карточка клиента (внешний источник).Индекс платежной дисциплины Значение",
    "Карточка клиента (внешний источник).Индекс финансового риска Значение"
]

# столбцы бинарных значений
bin_cols = [
    "Находится в реестре МСП",
    "Грузоотправитель", "Грузополучатель",
]

# столбцы идентификаторов
id_cols = ["ID"]

In [71]:
dss_paths = [
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Владимирская область.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Кировская область.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Нижегородская область.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Республика Марий Эл.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Республика Мордовия.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Республика Татарстан.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Республика Удмуртия.xls",
    r"РЖД train/Выгрузка_маркетинговые списки/МС_Республика Чувашия.xls"
]

for path in dss_paths:
    # чтение датасета
    ds = pd.read_excel(path)
    
    # удаление некоторых столбцов
    ds = ds.drop(drop_cols, axis=1, errors="ignore")
    
    # замена столбцов категорий столбцом чисел
    for col in cat_cols:
        ds[col] = ds[col].str.lower()
        ds[col] = ds[col].str.strip()
        
        le = LabelEncoder()
        mean_value = le.fit_transform(ds[col].dropna()).mean()
        
        le.fit(ds[col].dropna())
        ds[col] = ds[col].apply(lambda x: le.transform([x])[0] if pd.notna(x) else mean_value)
        ds[col] = ds[col].astype(np.int64)
    
    # очищение ОКВЭД2.Код от точек
    ds["ОКВЭД2.Код"] = ds["ОКВЭД2.Код"].str.replace('.', '')
    ds["ОКВЭД2.Код"] = ds["ОКВЭД2.Код"].astype(np.float64)
    
    # замена неизвестных значений на среднее из всех известных и преобразование в int
    for col in num_cols:
        mean_value = ds[col].dropna().mean()
        
        ds[col] = ds[col].apply(lambda x: mean_value if pd.isnull(x) else x)
        ds[col] = ds[col].astype(np.int64)
    
    # замена значений "Да" на 1, "Нет" на 0
    for col in bin_cols:
        ds[col] = ds[col].apply(lambda x: x if pd.isnull(x) else (0 if x == "Нет" else 1))
    
    # замена значений ЕЛС действующего на условие его присутствия
    ds["ЕЛС действующий"] = ds["ЕЛС действующий"].astype("string")
    ds["ЕЛС действующий"] = ds["ЕЛС действующий"].apply(lambda x: (0 if (pd.isnull(x) or x.strip() == '') else 1))
    
    # сохранение итогового датасета
    ds.to_csv(path[:-4] + "_Преобразованный.csv")